In [26]:
import os
import io
from random import seed
from random import randrange
import numpy as np
import pandas as pd
from pandas import DataFrame
from collections import Counter
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
path = '/content/drive/MyDrive/data/emails.csv'
all_data = pd.read_csv(path)
print(all_data.shape)
# all_data.head(5)

(5172, 3002)


In [3]:
all_data.describe()

,the,to,ect,and,for,of,a,you,hou,in,on,is,this,enron,i,be,that,will,have,with,your,at,we,s,are,it,by,com,as,from,gas,or,not,me,deal,if,meter,hpl,please,re,...,bold,catch,performing,accepted,matters,batch,continuing,winning,symbol,offsystem,decisions,produced,ended,greatest,degree,solmonson,imbalances,fall,fear,hate,fight,reallocated,debt,reform,australia,plain,prompt,remains,ifhsc,enhancements,connevey,jay,valued,lay,infrastructure,military,allowing,ff,dry,Prediction
count,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,...,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.00000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000
mean,6.640565,6.188128,5.143852,3.075599,3.124710,2.627030,55.517401,2.466551,2.024362,10.600155,10.935808,5.386118,1.388631,1.335267,45.857889,3.229312,0.924401,0.850928,0.804718,0.939675,0.814385,6.932328,1.978732,41.811872,1.408546,4.501160,0.657386,1.767208,4.807425,0.813998,0.617363,7.749033,0.838167,5.332367,0.734532,1.225251,0.538090,0.635151,0.627804,9.109049,...,0.005994,0.005607,0.004254,0.004254,0.004640,0.008701,0.005994,0.006381,0.008894,0.00406,0.004060,0.005414,0.070766,0.004060,0.008701,0.006187,0.005800,0.017208,0.006381,0.015855,0.005800,0.004640,0.006381,0.005027,0.006961,0.037510,0.008894,0.004060,0.005220,0.005607,0.005027,0.012568,0.010634,0.098028,0.004254,0.006574,0.004060,0.914733,0.006961,0.290023
std,11.745009,9.534576,14.101142,6.045970,4.680522,6.229845,87.574172,4.314444,6.967878,19.281892,17.999402,9.144407,2.849708,4.570525,77.832221,5.045836,2.084255,2.065738,1.444839,2.036278,2.001731,12.949766,3.719254,66.530032,2.583277,8.377237,1.247460,11.002626,7.568755,1.657193,1.844452,12.538043,1.771383,8.256332,1.930954,2.375526,1.945471,1.516908,1.130827,14.494729,...,0.118664,0.077224,0.073462,0.070781,0.067969,0.210261,0.077195,0.123437,0.110895,0.08675,0.066569,0.080906,0.367953,0.066569,0.132372,0.087733,0.100112,0.191417,0.097135,0.140928,0.133259,0.073439,0.097135,0.073413,0.162025,0.239546,0.099885,0.066569,0.084428,0.192108,0.105788,0.199682,0.116693,0.569532,0.096252,0.138908,0.072145,2.780203,0.098086,0.453817
min,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,12.000000,0.000000,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,8.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,8.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000

In [28]:
all_data.isnull().sum()

Email No.     0
the           0
to            0
ect           0
and           0
             ..
military      0
allowing      0
ff            0
dry           0
Prediction    0
Length: 3002, dtype: int64

In [ ]:
sample = all_data.iloc[:,1:]

n_sample = (len(sample))*2
while len(sample) < n_sample:
  index = randrange(len(sample))
  sample_t = sample.iloc[index]
  sample = sample.append(sample_t, ignore_index=True)
print(len(sample))

In [42]:
len_s_1 = int(len(sample)/3)
len_s_2 = int(2 * len(sample)/3)

X = sample.iloc[:,:-1].values
Y = sample.iloc[:,-1].values
# # Split sets to Train & Test sets for part 1
trainX, testX, trainY, testY = train_test_split(X, Y, test_size=0.2)
# Make 3 set from dataset for Bagging approach
X1 = X[:len_s_1,:]
Y1 = Y[:len_s_1]
X2 = X[len_s_1:len_s_2, :]
Y2 = Y[len_s_1:len_s_2]
X3 = X[len_s_2:, :]
Y3 = Y[len_s_2:]
# Split sets to Train & Test sets for part 2
train_x1, test_x1, train_y1, test_y1 = train_test_split(X1,Y1,test_size = 0.2)
train_x2, test_x2, train_y2, test_y2 = train_test_split(X2,Y2,test_size = 0.2)
train_x3, test_x3, train_y3, test_y3 = train_test_split(X3,Y3,test_size = 0.2)
test_x = np.concatenate((test_x1, test_x2, test_x3), axis=None)
test_y = np.concatenate((test_y1, test_y2, test_y3), axis=None)
print(train_x1.shape)
print(train_y2.shape)
print(test_y3.shape)
print(test_y.shape)

(2758, 3000)
(2758,)
(690,)
(2070,)


Bayesian Network

In [43]:
# Calculate & return probability of set
def probability(subset):

  sum_s = 0
  sum_ns = 0
  for i in subset:
      if i == 0:
          sum_s += 1
      else:
          sum_ns += 1
  prob_s = sum_s/len(subset)
  prob_ns = sum_ns/len(subset)
  # print(f"Spam Probability in sample set is {prob_s*100:.2f}%")
  # print(f"Not Spam Probability in sample set is {prob_ns*100:.2f}%")
  return prob_s, prob_ns

P(s|w) = {P(w|s)*P(s)} / {P(w|s)*P(s) + P(w|ns)*P(ns)}

In [44]:
def bayesian(trainx, trainy, testx, testy):

  # W given S & W given NS
  ### Calulate number of words in each Spam mail & Ham mail(NotSpam)
  wi_given_s = [0 for x in trainx[0]]
  wi_given_ns = [0 for x in trainx[0]]
  s_count = 0
  ns_count = 0

  for i,j in zip(trainx, trainy):
    if j == 0:
      s_count += sum(i)
      wi_given_s += i
    else:
      ns_count += sum(i)
      wi_given_ns += i

  # Probability of each word
  wi_given_s = wi_given_s / s_count
  # print(max(wi_given_s))
  wi_given_ns = wi_given_ns / ns_count
  # print(max(wi_given_ns))

  prob_s, prob_ns = probability(trainy)
  ### Calculate P(w|s)P(s) + P(w|ns)*P(ns)
  prob_wi = []
  for ps, pns in zip(wi_given_s, wi_given_ns):
    p_wi = (ps * prob_s) + (pns * prob_ns)
    prob_wi.append(p_wi)
  # S given W & NS given W
  s_given_wi = []
  for si, sj in zip(wi_given_s, prob_wi):
    ap = (si * prob_s)/sj
    s_given_wi.append(ap)
  # print(max(s_given_wi))

  ns_given_wi = []
  for nsi, nsj in zip(wi_given_ns, prob_wi):
    apn = (nsi * prob_ns)/nsj
    ns_given_wi.append(apn)
  # print(max(ns_given_wi))

  # Classification (Prediction)
  # P(S|W1)*P(S|W2)*....
  predict = 0
  acc_num = 0
  nb_pred = []

  for xt, yt in zip(testx, testy):
    sx=0
    nsx=0
    # Use LOG for small probability  
    for xxt, sgx in zip(xt, s_given_wi):
      if sgx != 0:
        sx += np.log10(sgx)
    for xxtn, nsgx in zip(xt, ns_given_wi):
      if nsgx != 0:
        nsx += np.log10(nsgx)
    if sx >= nsx:
      predict = 1
    else:
      predict = 0
    nb_pred.append(predict)
    if yt == predict:
      acc_num += 1
  accuracy = acc_num / len(testy)
  print(f"Bayesian Classification accuracy is {accuracy*100:.2f}%")
  return nb_pred


For part 1



In [45]:
nb_pred = bayesian(trainX, trainY, testX, testY)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: RuntimeWarning: invalid value encountered in double_scalars


Bayesian Classification accuracy is 72.40%


For Part 2

In [46]:
nb_pred_1 = bayesian(train_x1, train_y1, test_x1, test_y1)
nb_pred_2 = bayesian(train_x2, train_y2, test_x2, test_y2)
nb_pred_3 = bayesian(train_x3, train_y3, test_x3, test_y3)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: RuntimeWarning: invalid value encountered in double_scalars


Bayesian Classification accuracy is 70.87%
Bayesian Classification accuracy is 70.72%
Bayesian Classification accuracy is 70.87%


For Suport Vector Machine

In [47]:
class SVM:
  def __init__(self, learning_rate=0.001, n_iters=250):
    self.lr = learning_rate
    self.n_iters = n_iters
    self.w = None
    self.b = None

  def fit(self, X, y):
    n_samples, n_features = X.shape
    y_ = np.where(y <= 0, -1, 1)   
    self.w = np.zeros(n_features)
    self.b = 0
    for _ in range(self.n_iters):
      for idx, x_i in enumerate(X):
        # Compute correct place
        condition = y_[idx] * (np.dot(x_i, self.w) - self.b) >= 1
        # Update Weight & Bias
        if condition:
          self.w -= self.lr * self.w
        else:
          self.w -= self.lr * (self.w - np.dot(x_i, y_[idx]))
          self.b -= self.lr * y_[idx]

  def predict(self, X, y):
    cnt = 0
    # Predict result based on Weight & Bias
    approx = np.dot(X, self.w) - self.b
    pre = np.sign(approx)
    for ind, para in enumerate(pre):
      # On Margin=0 & Spam=1 & NotSpam=-1
      if para == -1:
        pre[ind] = 0
    diff = pre - y
    for p in diff:
      if p == 0:
        cnt += 1
               
    print(f"SVM Classification acuuracy is {(cnt/len(y))*100:.2f}%")
    return pre

In [48]:
clf = SVM()

clf.fit(train_x1, train_y1)
svm_pred_1 = clf.predict(test_x1, test_y1)
clf.fit(train_x2, train_y2)
svm_pred_2 = clf.predict(test_x2, test_y2)
clf.fit(train_x3, train_y3)
svm_pred_3 = clf.predict(test_x3, test_y3)


SVM Classification acuuracy is 80.58%
SVM Classification acuuracy is 73.33%
SVM Classification acuuracy is 65.36%


K_Nearest Neighbor

In [49]:
def euclidean_distance(x1, x2):
  return np.sqrt(np.sum((x1 - x2)**2))

class KNN:
  # KNN has not fitting
  def fit(self, X, y):
    self.X_train = X
    self.y_train = y

  def predict(self, X, Y):
    y_pred = [self._predict(x) for x in X]
    diff = list(Y - y_pred)
    acc = (diff.count(0) / len(Y))*100
    print(f"custom KNN classification accuracy is {acc:.2f}%")
    # print(y_pred)
    return y_pred

  def _predict(self, x):
    # Compute distances between x and all examples in the training set
    distances = [euclidean_distance(x, x_train) for x_train in self.X_train]
    # Sort by distance and return indices of the first k neighbors
    k_idx = np.argsort(distances)[:2]
    # Extract the labels of the k nearest neighbor training samples
    k_neighbor_labels = [self.y_train[i] for i in k_idx]  
    # return the most common class label
    most_common = Counter(k_neighbor_labels).most_common(1)
    return most_common[0][0]


In [50]:
clf = KNN()

clf.fit(train_x1, train_y1)
knn_pred_1 = clf.predict(test_x1, test_y1)
clf.fit(train_x2, train_y2)
knn_pred_2 = clf.predict(test_x2, test_y2)
clf.fit(train_x3, train_y3)
knn_pred_3 = clf.predict(test_x3, test_y3)

custom KNN classification accuracy is 85.22%
custom KNN classification accuracy is 89.13%
custom KNN classification accuracy is 93.19%


Ensemble Learning with Max-Voting

In [54]:
### Ensemble Learning (Bagging)
max_vote_pred = []
counter = 0
final_pred_1 = [nb_pred_1, svm_pred_1, knn_pred_1]
final_pred_2 = [nb_pred_2, svm_pred_2, knn_pred_2]
final_pred_3 = [nb_pred_3, svm_pred_3, knn_pred_3]

# Find max class for each email
def max_vote(final_pred):
  sum_pred = []
  sum_pred = [sum(i) for i in zip(*final_pred)]
  for p in sum_pred:
    if p >= 2:
      max_vote_pred.append(1)
    else:
      max_vote_pred.append(0)

max_vote(final_pred_1)
max_vote(final_pred_2)
max_vote(final_pred_3)
# Accuracy
f_diff = test_y - max_vote_pred
for fd in f_diff:
  if fd == 0:
    counter += 1
# print(counter)
final_acc = counter / (len(test_y))
print(f"Ensemble learning's accuracy is {final_acc*100:.2f}%")

Ensemble learning's accuracy is 82.85%
